`DESCRIÇÃO DOS ATRIBUTOS:`
            
1. Day_of_Week {'Thursday', 'Monday', 'Sunday', 'Tuesday', 'Friday', 'Wednesday','Saturday'}  

2. Junction_Detail {'T or staggered junction', 'Crossroads','Not at junction or within 20 metres', 'Roundabout','Mini-roundabout', 'More than 4 arms (not roundabout)','Private drive or entrance', 'Slip road', 'Other junction'}           

3. Accident_Injuries {'Serious', 'Slight', 'Fatal'}    

4. Light_Conditions {'Day', 'Darkness - lights lit', 'Darkness - lighting unknown', 'Darkness - lights unlit', 'Darkness - no lighting'}       

5. Number_of_Casualties {1, 11,  3,  2,  4,  5,  6, 10,  8,  7,  9, 13, 12, 14, 16, 18, 15, 19, 21, 17, 27, 48, 40, 24, 26, 22, 42, 43}      

6. Number_of_Vehicles {2,  1,  3,  4,  5,  9,  6,  8,  7, 10, 16, 13, 14, 11, 12, 32, 19}   

7. Road_Surface_Conditions {'Dry', 'Wet or damp', 'Frost or ice', 'Snow','Flood over 3cm. deep'}

8. Road_Type {'One way street', 'Single carriageway', 'Dual carriageway','Roundabout', 'Slip road'}         

9. Speed_limit {30, 20, 50, 40, 70, 60, 10, 15}     

10. Hour_of_Accident {3-6, 6-9, 9-12, 12-15, 15-18, 18-21, 21-24}

11. Urban_or_Rural_Area {'Urban', 'Rural'}   

12. Weather_Conditions {'Fine no high winds', 'Other', 'Raining no high winds','Snowing no high winds', 'Fine + high winds', 'Raining + high winds', 'Snowing + high winds', 'Fog or mist'}   

13. Vehicle_Type {'Car', 'Taxi', 'Motorcycle', 'Van', 'Other', 'Bus', 'Pedal cycle','Ridden horse'}

In [4]:
!pip install pydotplus
!pip install dtreeviz

In [ ]:
import pandas as pd
import numpy as np

from sklearn import datasets, tree
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.feature_extraction import DictVectorizer # converte dicionários Python em matrizes NumPy ou em matrizes esparsas  # ?
from sklearn.preprocessing import LabelEncoder        # converte rótulos de classes categóricas em números                     # ?

In [6]:
dados = pd.read_csv("/content/car_accidentOFICIAL.csv")

### Transformação de dados               # ?

Inicialmente os atributos são convertidos para o formato de dicionário.
Em seguida os atributos são transformados em binário, para cada par (key, value) do dicionário.

<code>DictVectorizer(dtype=class ‘numpy.float64’, separator=’=’, sparse=True, sort=True)</code>

A classe é convertida para labels únicos sequenciais.

<code>
 le = preprocessing.LabelEncoder()
  
 le.fit(dados)
</code>


### Carga dos dados e particionamento das bases de treinamento e teste

`train_test_split(X, y) -- PARTICIONA A BASE DE DADOS EM BASES DE TREINAMENTO E TESTE.`

Por padrão, 75% da base é utilizada para treinamento e 25% para testes.

No código a seguir, usaremos 15% para teste e 85% para treinamento:



In [22]:
# Divide os dados em conjunto de recursos (X) e conjunto de rótulos (y)

# Aqui 'X' são os recursos (todas as colunas exceto a que você quer prever) e 'y' é o rótulo (a coluna que você quer prever)
X = dados.drop(columns=['Accident_Injuries'])  # Deixe de fora a coluna que você quer prever
y = dados['Accident_Injuries'] # Essa é a coluna que queremos prever

# Divide os dados em conjunto de treinamento e conjunto de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.15)



*O conjunto de recursos X (ou conjunto de características) é composto pelos atributos ou variáveis que são usados para prever o resultado.
O conjunto de recursos é frequentemente representado como uma matriz, onde cada linha corresponde a uma amostra e cada coluna corresponde a uma característica.

*O conjunto de rótulos y (ou conjunto de alvos) é composto pelas variáveis que estamos tentando prever ou classificar.
Em problemas de classificação, os rótulos representam as categorias ou classes às quais as amostras pertencem. O conjunto de rótulos geralmente é representado como um vetor.





Dado que "Accident_Injuries" foi o dado usado para a pergunta orientada a dados, isso pode influenciar a escolha dos atributos para a construção da árvore de decisão. Nesse caso, "Accident_Injuries" seria a variável dependente (y), pois é o que você está tentando prever ou entender melhor.

Portanto, "Accident_Injuries" seria a variável alvo (y) e os outros atributos listados seriam as variáveis independentes (X). Você usaria esses atributos para construir a árvore de decisão e entender quais características estão mais relacionadas com o número de lesões em acidentes de trânsito.




### Indução do Modelo


Os três passos para indução de um modelo são:

1.   Instanciar o modelo: ``` DecisionTreeClassifier()```
2.   Treinar o modelo: ```fit()```
3.   Testar o modelo: ```predict()```



In [25]:
tree_accident = DecisionTreeClassifier(random_state=0, criterion='entropy', class_weight=[{0: 1, 1: 1}])

tree_accident = tree_accident.fit(X_train, y_train)
print("Acurácia (base de treinamento):", tree_accident.score(X_train, y_train))

y_pred = tree_accident.predict(X_test)
print("Acurácia de previsão:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=accident.target_names))

cnf_matrix = confusion_matrix(y_test, y_pred)
cnf_table = pd.DataFrame(data=cnf_matrix, index=accident.target_names, columns=[x + "(prev)" for x in accident.target_names])
print(cnf_table)


ValueError: could not convert string to float: '200950JN2P015'

In [26]:
# Criar e treinar o modelo de árvore de decisão
tree_accident = DecisionTreeClassifier(random_state=0, criterion='entropy', class_weight={0: 1, 1: 1})
tree_accident.fit(X_train, y_train)

# Avaliar o modelo
train_accuracy = tree_accident.score(X_train, y_train)
print("Acurácia (conjunto de treinamento):", train_accuracy)

y_pred = tree_accident.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("Acurácia (conjunto de teste):", test_accuracy)

# Relatório de classificação
print(classification_report(y_test, y_pred))

# Matriz de confusão
cnf_matrix = confusion_matrix(y_test, y_pred)
cnf_table = pd.DataFrame(data=cnf_matrix, index=['Classe 0', 'Classe 1'], columns=['Classe 0 (prev)', 'Classe 1 (prev)'])
print(cnf_table)

ValueError: could not convert string to float: '200950JN2P015'

O erro ValueError: could not convert string to float: '200950JN2P015' indica que há uma string no seu conjunto de dados que não pode ser convertida para um tipo float. Isso geralmente acontece quando há dados não numéricos em colunas que deveriam conter apenas números.

### Exibição da árvore de decisão

(#)8? - isso aqui são 3 exemplos! você pega 1, modifica e apaga os outros 2.

In [ ]:
from dtreeviz.trees import *

viz = dtreeviz(tree_iris,
              X_train,
              y_train,
              target_name="espécie",
              feature_names=iris.feature_names,
              class_names=["setosa", "versicolor", "virginica"])

viz.view()

In [ ]:
from dtreeviz.trees import *

viz = dtreeviz(sonar_tree,
              X_train,
              y_train,
              target_name="Classe",
              feature_names=np.arange(0,60),
              class_names=["mina", "rocha"])

viz.view()

In [ ]:
from dtreeviz.trees import *

viz = dtreeviz(treeClima,
              X_train,
              y_train,
              target_name='Comprar',
               feature_names=vect.feature_names_,
              class_names=["sim", "nao"],
              fancy=True)

viz.view()

 (#)9? - isso aqui são 3 exemplos! você pega 1, modifica e apaga os outros 2.

In [ ]:
import pydotplus
# Create DOT data
dot_data = tree.export_graphviz(tree_iris, out_file=None,
                                #proportion=True,
                                rounded =True,
                                filled=True,
                                feature_names=iris.feature_names,
                                class_names=["setosa", "versicolor", "virginica"])

# Draw graph
graph = pydotplus.graph_from_dot_data(dot_data)

# Show graph
Image(graph.create_png())


In [ ]:
import pydotplus
# Create DOT data
dot_data = tree.export_graphviz(sonar_tree, out_file=None,
                                proportion=False,
                                rounded =True,
                                filled=True,
                                feature_names=np.arange(0,60),
                                class_names=["mina", "rocha"])

# Draw graph
graph = pydotplus.graph_from_dot_data(dot_data)

# Show graph
Image(graph.create_png())


In [ ]:
import pydotplus
# Create DOT data
dot_data = tree.export_graphviz(treeClima, out_file=None,
                                feature_names=vect.feature_names_,
                                rounded =True,
                                filled=True,
                                class_names=["sim", "nao"])

# Draw graph
graph = pydotplus.graph_from_dot_data(dot_data)

# Show graph
Image(graph.create_png())
